In [17]:
#PATH_data = f"../abyss_data/"
#Mount external drive: sudo mount -t drvfs D: /mnt/d
PATH_data = f"/mnt/d/abyss_data/"
PATH_GENO = f"{PATH_data}genotype"
PATH_SUMSTAT = f"{PATH_data}sumstat/mono_PCA/"
PATH_GENO_segmented = f"{PATH_GENO}/segmented/"
PATH_GENO_mini_dim = f"{PATH_GENO}/mini_dim/"
PATH_GENO_maf = f"{PATH_GENO}/maf/"
PATH_PHENO = f"{PATH_data}phenotype/"
PATH_PHENO_ancestry = f"{PATH_PHENO}/ancestry/all.panel"
PATH_PLOTS = f"{PATH_data}plots/"
PATH_PARAMS = './hyper_params_to_test_AF.txt'

In [18]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
from collections import Counter
import os
from os import listdir
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

In [19]:
effect = 3
chrom = 21
maf_causal_snp = 0.2
sit = "strat_causal"

In [20]:
path_genos = PATH_GENO_segmented + f"chrom_{chrom}/unshuffled/"


In [22]:
onlyfiles_input = [f for f in listdir(
    path_genos) if maf_causal_snp + maf_causal_snp/5 > float(f.split('_')[5]) > maf_causal_snp - maf_causal_snp/5]


In [6]:
popped = onlyfiles_input[round(len(onlyfiles_input)/2)]
snp_nr = 0

In [7]:
popped

'19_5_800_19_maf_0.2_0.25.pkl'

In [8]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}")

mkdir: cannot create directory ‘/mnt/d/abyss_data/sumstat/mono/maf_causal_0.2’: No such file or directory


256

In [9]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}")


mkdir: cannot create directory ‘/mnt/d/abyss_data/sumstat/mono/maf_causal_0.2/21’: No such file or directory


256

In [10]:
PATH_mono = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}"


In [12]:
try:
  causal_file = [f for f in listdir(
      PATH_mono) if f.startswith("from_chrom")][0]
  snps = pd.read_pickle(f"{PATH_mono}/{causal_file}")
except Exception as e:
  print(e)
  snps = pd.read_pickle(path_genos+popped)
  snps.to_pickle(f"{PATH_mono}/from_chrom_{chrom}_{popped}")
  snp = snps.iloc[:, snp_nr]
  name = snps.columns[snp_nr]
  snp.to_pickle(f"{PATH_mono}/causal_snp_{name}.pkl")

In [13]:
causal_file = [f for f in listdir(PATH_mono) if f.startswith("from_chrom")][0]

In [14]:
humans = pd.read_pickle(f"{PATH_data}sumstat/humans_no_causal.pkl")


In [15]:
snps = pd.read_pickle(f"{PATH_mono}/{causal_file}")

name = snps.columns[snp_nr]

snp_effect = snps.iloc[:, snp_nr] * effect
#snp_effect = snps.iloc[:, snp_nr] * (effect/float(maf.split('_')[0]))

humans['no_strat_causal'] = humans['no_strat_no_causal']
humans['strat_causal'] = humans['strat_no_causal']

humans['no_strat_causal'] = humans['no_strat_causal'] + snp_effect
humans['strat_causal'] = humans['strat_causal'] + snp_effect

In [16]:
print(Counter(snp_effect).keys())

dict_keys([3, -3, 0])


In [656]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}/{sit}")

0

In [657]:
path_output = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}/{sit}/"


In [658]:
pheno = np.array(humans[sit])

In [659]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [660]:
onlyfiles_input = [f for f in listdir(path_genos)]

In [661]:
for file in onlyfiles_input:
    path_file = path_genos + '/' + file
    genos = pd.read_pickle(path_file)    
    X = genos
    y = pheno
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    sumstat = pd.DataFrame(data={
        'snp': list(est2.pvalues.index)[1:],
        'coef': list(est2.params)[1:],
        'std_err': list(est2.bse)[1:],
        't': list(est2.tvalues)[1:],
        'P': list(est2.pvalues[1:]),
        '0.025': list(est2.conf_int()[0])[1:],
        '0.975': list(est2.conf_int()[1])[1:]
    })
    sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
    sumstat['POS'] = [i.split(':', 1)[0]
                      for i in list(sumstat['snp'].str[3:-5])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    sumstat['-logp'] = - np.log10(sumstat['P'])
    sumstat.to_pickle(f"{path_output}/{file}")


In [662]:
path_sumstat = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}/"

In [663]:
chrom_files = [f for f in listdir(path_sumstat) if f != causal_file]
chrom_files = [f for f in chrom_files if f != f"causal_snp_{name}.pkl"]
chrom_files = [f for f in chrom_files if f.split("_")[0] != f"GWAS"]


In [664]:
chrom_files

['21', '22']

In [665]:
sumstat_array = []
for chrom_file in chrom_files:
    sumstat_files = [f for f in listdir(path_sumstat + chrom_file + '/' + sit + "/")]
    # first do it with the first one
    path_file = path_sumstat + chrom_file + '/' + sit + '/' + sumstat_files[0]
    sumstats = pd.read_pickle(path_file)
    for sumstat_file in sumstat_files[1:]:
        path_file = path_sumstat + chrom_file + '/' + sit + '/' + sumstat_file
        temp = pd.read_pickle(path_file)
        sumstats = pd.concat([sumstats,temp])
    sumstat_array.append(sumstats)

sumstats = sumstat_array[0]
for sumstat in sumstat_array[1:]:
    sumstats = pd.concat([sumstats,sumstat]) 

In [666]:
sumstats = sumstats.reset_index(drop=True)

In [667]:
running_pos = 0
cumulative_pos = []
for chrom, group_df in sumstats.groupby('#CHROM'):
    cumulative_pos.append(group_df['POS'] + running_pos)
    running_pos += group_df['POS'].max()

sumstats['cumulative_pos'] = pd.concat(cumulative_pos)
sumstats['SNP number'] = sumstats.index

In [668]:
causal_chrom = causal_file.split("_")[2]

In [669]:
if sit == 'strat_causal':
  title = f"One causal snp with AF of {maf_causal_snp} on chrom {causal_chrom} with a population dependent phenotype"
elif sit == 'strat_no_causal':
  title = f"No causal snp but with a population dependent phenotype"
elif sit == "no_strat_causal":
  title = f"One causal snp with AF of {maf_causal_snp} on chrom {causal_chrom} with no population dependent phenotype"
else:
  title = f"No causal snp and no population dependent phenotype"
sit

'no_strat_no_causal'

In [670]:
title

'No causal snp and no population dependent phenotype'

In [671]:
sumstats

,snp,coef,std_err,t,P,0.025,0.975,#CHROM,POS,-logp,cumulative_pos,SNP number
0,21:10892322:A:G_A,-1.626230,3.015333,-0.539320,0.589749,-7.541157,4.288697,21,10892322,0.229332,10892322,0
1,21:47368531:A:G_G,-0.181777,2.304512,-0.078879,0.937140,-4.702345,4.338791,21,47368531,0.028195,47368531,1
2,21:32892189:A:G_G,-1.043117,2.462575,-0.423588,0.671930,-5.873743,3.787509,21,32892189,0.172676,32892189,2
3,21:15792338:A:C_C,-0.608730,0.438467,-1.388315,0.165256,-1.468834,0.251374,21,15792338,0.781842,15792338,3
4,21:25645657:G:T_T,-11.173373,8.176706,-1.366488,0.172000,-27.212932,4.866187,21,25645657,0.764473,25645657,4
...,...,...,...,...,...,...,...,...,...,...,...,...
479995,22:31587218:C:T_C,1.163789,1.183898,0.983015,0.325765,-1.158556,3.486135,22,31587218,0.487096,79706852,479995
479996,22:37377737:C:T_T,-6.205272,4.396451,-1.411428,0.158334,-14.829390,2.418847,22,37377737,0.800425,85497371,479996
479997,22:30678530:A:G_A,-0.801674,2.195977,-0.365065,0.715117,-5.109323,3.505975,22,30678530,0.145623,78798164,479997
479998,22:27063804:C:G_G,0.679335,0.544600,1.247402,0.212453,-0.388957,1.747626,22,27063804,0.672738,75183438,479998


In [672]:
sumstats['causality'] = np.zeros(sumstats.shape[0])

In [673]:
sumstats['causality'].mask(sumstats['snp'] == name, 1, inplace=True)


In [674]:
sumstats = sumstats.sort_values(by=['causality'])


In [675]:
sns.relplot(
    data=sumstats,
    x='cumulative_pos',
    y='-logp',
    aspect=2,
    hue='#CHROM',
    style = "causality",
    palette='dark'
).set(title=title)
plt.savefig(f"{path_sumstat}/GWAS_mono_{sit}.png")
plt.savefig(f"./GWAS_Common.png")